In [1]:
import h5py
import pandas as pd
import shutil
import tensorflow as tf

2023-06-07 20:45:30.567417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 20:45:31.108747: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-07 20:45:31.108795: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-06-07 20:45:31.108799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(10*1000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
file_dir = f'/data/HSC/HSC_v6/step2/127x127/'
original_file_name = f'five_band_image127x127_with_metadata.hdf5'
new_file_name = 'five_band_image127x127_with_metadata_with_morphology.hdf5'

In [4]:
g_df = pd.read_csv('/data/HSC/HSC_v6/step1/g_band_sextractor/shape_parameters.csv')
r_df = pd.read_csv('/data/HSC/HSC_v6/step1/r_band_sextractor/shape_parameters.csv')
i_df = pd.read_csv('/data/HSC/HSC_v6/step1/i_band_sextractor/shape_parameters.csv')
z_df = pd.read_csv('/data/HSC/HSC_v6/step1/z_band_sextractor/shape_parameters.csv')
y_df = pd.read_csv('/data/HSC/HSC_v6/step1/y_band_sextractor/shape_parameters.csv')

In [5]:
g_df = g_df[['object_id', 'specz', 'X_IMAGE', 'Y_IMAGE', 'g_cmodel_mag', 'PETRO_RADIUS', 'ISOAREA_IMAGE', 'A_IMAGE', 'B_IMAGE', 'THETA_IMAGE', 'MU_MAX', 'ELLIPTICITY', 'FLUX_RADIUS', 'SPHEROID_SERSICN', 'NUMBER_IN_CENTER_30PX_DIAMETER', 'NUMBER_IN_CENTER_20PX_DIAMETER', 'NUMBER_IN_CENTER_10PX_DIAMETER']]
r_df = r_df[['object_id', 'specz', 'X_IMAGE', 'Y_IMAGE', 'r_cmodel_mag', 'PETRO_RADIUS', 'ISOAREA_IMAGE', 'A_IMAGE', 'B_IMAGE', 'THETA_IMAGE', 'MU_MAX', 'ELLIPTICITY', 'FLUX_RADIUS', 'SPHEROID_SERSICN', 'NUMBER_IN_CENTER_30PX_DIAMETER', 'NUMBER_IN_CENTER_20PX_DIAMETER', 'NUMBER_IN_CENTER_10PX_DIAMETER']]
i_df = i_df[['object_id', 'specz', 'X_IMAGE', 'Y_IMAGE', 'i_cmodel_mag', 'PETRO_RADIUS', 'ISOAREA_IMAGE', 'A_IMAGE', 'B_IMAGE', 'THETA_IMAGE', 'MU_MAX', 'ELLIPTICITY', 'FLUX_RADIUS', 'SPHEROID_SERSICN', 'NUMBER_IN_CENTER_30PX_DIAMETER', 'NUMBER_IN_CENTER_20PX_DIAMETER', 'NUMBER_IN_CENTER_10PX_DIAMETER']]
z_df = z_df[['object_id', 'specz', 'X_IMAGE', 'Y_IMAGE', 'z_cmodel_mag', 'PETRO_RADIUS', 'ISOAREA_IMAGE', 'A_IMAGE', 'B_IMAGE', 'THETA_IMAGE', 'MU_MAX', 'ELLIPTICITY', 'FLUX_RADIUS', 'SPHEROID_SERSICN', 'NUMBER_IN_CENTER_30PX_DIAMETER', 'NUMBER_IN_CENTER_20PX_DIAMETER', 'NUMBER_IN_CENTER_10PX_DIAMETER']]
y_df = y_df[['object_id', 'specz', 'X_IMAGE', 'Y_IMAGE', 'y_cmodel_mag', 'PETRO_RADIUS', 'ISOAREA_IMAGE', 'A_IMAGE', 'B_IMAGE', 'THETA_IMAGE', 'MU_MAX', 'ELLIPTICITY', 'FLUX_RADIUS', 'SPHEROID_SERSICN', 'NUMBER_IN_CENTER_30PX_DIAMETER', 'NUMBER_IN_CENTER_20PX_DIAMETER', 'NUMBER_IN_CENTER_10PX_DIAMETER']]

In [6]:
g_df.rename(columns={'X_IMAGE': 'x_coord', 'Y_IMAGE': 'y_coord', 'PETRO_RADIUS': 'g_petro_rad', 'ISOAREA_IMAGE': 'g_isophotal_area', 'A_IMAGE': 'g_major_axis', 'B_IMAGE': 'g_minor_axis', 'THETA_IMAGE': 'g_pos_angle', 'MU_MAX': 'g_peak_surface_brightness', 'ELLIPTICITY': 'g_ellipticity', 'FLUX_RADIUS': 'g_half_light_radius', 'SPHEROID_SERSICN': 'g_sersic_index', 'NUMBER_IN_CENTER_30PX_DIAMETER': 'g_central_image_pop_15px_rad', 'NUMBER_IN_CENTER_20PX_DIAMETER': 'g_central_image_pop_10px_rad', 'NUMBER_IN_CENTER_10PX_DIAMETER': 'g_central_image_pop_5px_rad'}, inplace=True)
r_df.rename(columns={'X_IMAGE': 'x_coord', 'Y_IMAGE': 'y_coord', 'PETRO_RADIUS': 'r_petro_rad', 'ISOAREA_IMAGE': 'r_isophotal_area', 'A_IMAGE': 'r_major_axis', 'B_IMAGE': 'r_minor_axis', 'THETA_IMAGE': 'r_pos_angle', 'MU_MAX': 'r_peak_surface_brightness', 'ELLIPTICITY': 'r_ellipticity', 'FLUX_RADIUS': 'r_half_light_radius', 'SPHEROID_SERSICN': 'r_sersic_index', 'NUMBER_IN_CENTER_30PX_DIAMETER': 'r_central_image_pop_15px_rad', 'NUMBER_IN_CENTER_20PX_DIAMETER': 'r_central_image_pop_10px_rad', 'NUMBER_IN_CENTER_10PX_DIAMETER': 'r_central_image_pop_5px_rad'}, inplace=True)
i_df.rename(columns={'X_IMAGE': 'x_coord', 'Y_IMAGE': 'y_coord', 'PETRO_RADIUS': 'i_petro_rad', 'ISOAREA_IMAGE': 'i_isophotal_area', 'A_IMAGE': 'i_major_axis', 'B_IMAGE': 'i_minor_axis', 'THETA_IMAGE': 'i_pos_angle', 'MU_MAX': 'i_peak_surface_brightness', 'ELLIPTICITY': 'i_ellipticity', 'FLUX_RADIUS': 'i_half_light_radius', 'SPHEROID_SERSICN': 'i_sersic_index', 'NUMBER_IN_CENTER_30PX_DIAMETER': 'i_central_image_pop_15px_rad', 'NUMBER_IN_CENTER_20PX_DIAMETER': 'i_central_image_pop_10px_rad', 'NUMBER_IN_CENTER_10PX_DIAMETER': 'i_central_image_pop_5px_rad'}, inplace=True)
z_df.rename(columns={'X_IMAGE': 'x_coord', 'Y_IMAGE': 'y_coord', 'PETRO_RADIUS': 'z_petro_rad', 'ISOAREA_IMAGE': 'z_isophotal_area', 'A_IMAGE': 'z_major_axis', 'B_IMAGE': 'z_minor_axis', 'THETA_IMAGE': 'z_pos_angle', 'MU_MAX': 'z_peak_surface_brightness', 'ELLIPTICITY': 'z_ellipticity', 'FLUX_RADIUS': 'z_half_light_radius', 'SPHEROID_SERSICN': 'z_sersic_index', 'NUMBER_IN_CENTER_30PX_DIAMETER': 'z_central_image_pop_15px_rad', 'NUMBER_IN_CENTER_20PX_DIAMETER': 'z_central_image_pop_10px_rad', 'NUMBER_IN_CENTER_10PX_DIAMETER': 'z_central_image_pop_5px_rad'}, inplace=True)
y_df.rename(columns={'X_IMAGE': 'x_coord', 'Y_IMAGE': 'y_coord', 'PETRO_RADIUS': 'y_petro_rad', 'ISOAREA_IMAGE': 'y_isophotal_area', 'A_IMAGE': 'y_major_axis', 'B_IMAGE': 'y_minor_axis', 'THETA_IMAGE': 'y_pos_angle', 'MU_MAX': 'y_peak_surface_brightness', 'ELLIPTICITY': 'y_ellipticity', 'FLUX_RADIUS': 'y_half_light_radius', 'SPHEROID_SERSICN': 'y_sersic_index', 'NUMBER_IN_CENTER_30PX_DIAMETER': 'y_central_image_pop_15px_rad', 'NUMBER_IN_CENTER_20PX_DIAMETER': 'y_central_image_pop_10px_rad', 'NUMBER_IN_CENTER_10PX_DIAMETER': 'y_central_image_pop_5px_rad'}, inplace=True)

In [7]:
df = pd.merge(g_df, r_df, on=['object_id', 'specz'])
df = pd.merge(i_df, df, on=['object_id', 'specz'])
df = pd.merge(z_df, df, on=['object_id', 'specz'])
df = pd.merge(y_df, df, on=['object_id', 'specz'])

/tmp/ipykernel_2390189/3832309194.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'x_coord_y', 'y_coord_y'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(z_df, df, on=['object_id', 'specz'])


In [8]:
df

,object_id,specz,x_coord,y_coord,y_cmodel_mag,y_petro_rad,y_isophotal_area,y_major_axis,y_minor_axis,y_pos_angle,...,r_major_axis,r_minor_axis,r_pos_angle,r_peak_surface_brightness,r_ellipticity,r_half_light_radius,r_sersic_index,r_central_image_pop_15px_rad,r_central_image_pop_10px_rad,r_central_image_pop_5px_rad
0,36407046198803509,0.43602,60.6967,60.0606,17.614677,7.92,372.0,4.266,3.966,-82.86,...,5.447,4.986,72.23,-6.4936,0.085,6.856,1.502,2,1,1
1,36407046198804043,0.23209,60.7407,60.9607,16.851135,7.92,824.0,6.518,5.251,-38.78,...,9.150,7.210,-39.48,-7.0298,0.212,10.747,2.224,1,1,1
2,36407050493759629,0.56511,60.7617,60.2855,18.164425,9.24,399.0,4.771,3.885,-87.73,...,4.814,3.993,-85.69,-5.4887,0.171,7.051,1.620,1,1,1
3,36407050493773284,0.55775,60.8754,60.8286,19.115318,9.90,190.0,3.286,2.800,61.63,...,3.546,2.854,71.63,-5.3227,0.195,5.932,2.257,1,1,1
4,36407054788744371,0.44495,61.1102,60.2242,18.476147,6.60,233.0,3.420,3.023,-39.18,...,3.928,3.836,-47.82,-6.3640,0.023,4.704,1.359,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286396,74649168714401583,1.86393,61.6153,59.9096,20.932972,5.94,62.0,1.786,1.721,85.12,...,2.232,2.069,62.51,-6.1867,0.073,2.394,1.041,1,1,1
286397,74649168714403940,0.89864,59.3400,61.3752,21.601278,9.90,34.0,1.875,1.305,-28.82,...,2.398,2.007,-16.20,-4.0196,0.163,3.281,0.550,1,1,1
286398,74649168714420584,0.24194,59.4535,61.0472,16.965544,6.60,972.0,6.226,5.502,-52.62,...,6.638,5.815,-45.89,-7.7220,0.124,6.367,1.995,1,1,1
286399,74649168714426572,0.77584,61.3318,59.1751,21.360008,10.56,17.0,1.639,0.817,-88.91,...,3.169,2.163,67.10,-3.9291,0.318,4.182,0.558,1,1,1


In [9]:
df.drop(columns='specz', inplace=True)

In [10]:
with h5py.File(file_dir + new_file_name, 'r') as f:
    object_ids = list(f['object_id'][:])
object_ids_df = pd.DataFrame({'object_id': object_ids, 'index_pos': range(len(object_ids))})
merged_df = pd.merge(df, object_ids_df, on='object_id', how='inner')
sorted_df = merged_df.sort_values('index_pos')
sorted_df = sorted_df.drop('index_pos', axis=1)

In [11]:
sorted_df.columns

Index(['object_id', 'x_coord', 'y_coord', 'y_cmodel_mag', 'y_petro_rad',
       'y_isophotal_area', 'y_major_axis', 'y_minor_axis', 'y_pos_angle',
       'y_peak_surface_brightness', 'y_ellipticity', 'y_half_light_radius',
       'y_sersic_index', 'y_central_image_pop_15px_rad',
       'y_central_image_pop_10px_rad', 'y_central_image_pop_5px_rad',
       'x_coord_x', 'y_coord_x', 'z_cmodel_mag', 'z_petro_rad',
       'z_isophotal_area', 'z_major_axis', 'z_minor_axis', 'z_pos_angle',
       'z_peak_surface_brightness', 'z_ellipticity', 'z_half_light_radius',
       'z_sersic_index', 'z_central_image_pop_15px_rad',
       'z_central_image_pop_10px_rad', 'z_central_image_pop_5px_rad',
       'x_coord_y', 'y_coord_y', 'i_cmodel_mag', 'i_petro_rad',
       'i_isophotal_area', 'i_major_axis', 'i_minor_axis', 'i_pos_angle',
       'i_peak_surface_brightness', 'i_ellipticity', 'i_half_light_radius',
       'i_sersic_index', 'i_central_image_pop_15px_rad',
       'i_central_image_pop_10px_rad',

In [12]:
sorted_df.drop(columns=['object_id', 'g_cmodel_mag', 'r_cmodel_mag', 'i_cmodel_mag', 'z_cmodel_mag', 'y_cmodel_mag'], axis=1, inplace=True)

In [13]:
sorted_df.columns

Index(['x_coord', 'y_coord', 'y_petro_rad', 'y_isophotal_area', 'y_major_axis',
       'y_minor_axis', 'y_pos_angle', 'y_peak_surface_brightness',
       'y_ellipticity', 'y_half_light_radius', 'y_sersic_index',
       'y_central_image_pop_15px_rad', 'y_central_image_pop_10px_rad',
       'y_central_image_pop_5px_rad', 'x_coord_x', 'y_coord_x', 'z_petro_rad',
       'z_isophotal_area', 'z_major_axis', 'z_minor_axis', 'z_pos_angle',
       'z_peak_surface_brightness', 'z_ellipticity', 'z_half_light_radius',
       'z_sersic_index', 'z_central_image_pop_15px_rad',
       'z_central_image_pop_10px_rad', 'z_central_image_pop_5px_rad',
       'x_coord_y', 'y_coord_y', 'i_petro_rad', 'i_isophotal_area',
       'i_major_axis', 'i_minor_axis', 'i_pos_angle',
       'i_peak_surface_brightness', 'i_ellipticity', 'i_half_light_radius',
       'i_sersic_index', 'i_central_image_pop_15px_rad',
       'i_central_image_pop_10px_rad', 'i_central_image_pop_5px_rad',
       'x_coord_x', 'y_coord_x', 'g_

In [14]:
sorted_df

,x_coord,y_coord,y_petro_rad,y_isophotal_area,y_major_axis,y_minor_axis,y_pos_angle,y_peak_surface_brightness,y_ellipticity,y_half_light_radius,...,r_major_axis,r_minor_axis,r_pos_angle,r_peak_surface_brightness,r_ellipticity,r_half_light_radius,r_sersic_index,r_central_image_pop_15px_rad,r_central_image_pop_10px_rad,r_central_image_pop_5px_rad
67036,60.1847,60.8307,5.94,662.0,5.997,4.884,-14.93,-6.7931,0.186,6.874,...,6.324,5.102,-15.10,-5.3648,0.193,7.028,0.700,1,1,1
197850,60.1879,60.6818,6.60,487.0,4.610,4.399,83.26,-7.1644,0.046,5.703,...,4.797,4.397,-76.71,-5.7815,0.083,5.738,2.442,1,1,1
136714,60.3021,58.8913,5.94,2079.0,11.852,8.688,78.50,-6.9908,0.267,13.675,...,13.187,9.510,77.11,-6.3787,0.279,13.811,0.654,1,1,1
141728,60.3998,60.2224,6.60,559.0,4.790,4.508,68.80,-7.4040,0.059,5.604,...,4.824,4.342,36.89,-5.8784,0.100,5.461,2.313,1,1,1
143153,60.6733,60.5652,10.56,747.0,6.913,4.772,77.11,-7.3405,0.310,9.109,...,6.948,4.755,77.53,-6.1978,0.316,9.090,3.809,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283075,60.4239,59.7341,6.60,65.0,2.258,1.589,-45.29,-6.1487,0.296,2.970,...,2.014,1.589,-41.84,-4.2297,0.211,2.897,1.157,1,1,1
282895,46.0058,43.0710,10.56,11.0,1.104,0.745,15.20,-4.4395,0.325,2.885,...,1.218,1.021,-0.20,-3.2541,0.162,2.526,1.257,1,1,1
279179,78.4520,67.8434,10.56,83.0,2.845,2.044,-36.05,-6.1446,0.282,5.956,...,1.437,1.282,28.46,-3.6159,0.108,4.406,2.339,2,1,1
281025,59.9329,60.1444,10.56,11.0,1.025,0.828,36.75,-4.7882,0.193,2.656,...,1.710,1.527,-78.07,-3.6950,0.107,3.631,1.847,1,1,1


In [15]:
with h5py.File(file_dir + new_file_name, 'a') as f:
    column_names = sorted_df.columns.tolist()
    for column_name in column_names:
        column_data = sorted_df[column_name].values
        print(column_name)
        try:
            f[column_name] = column_data
        except:
            continue

x_coord
y_coord
y_petro_rad
y_isophotal_area
y_major_axis
y_minor_axis
y_pos_angle
y_peak_surface_brightness
y_ellipticity
y_half_light_radius
y_sersic_index
y_central_image_pop_15px_rad
y_central_image_pop_10px_rad
y_central_image_pop_5px_rad
x_coord_x
y_coord_x
z_petro_rad
z_isophotal_area
z_major_axis
z_minor_axis
z_pos_angle
z_peak_surface_brightness
z_ellipticity
z_half_light_radius
z_sersic_index
z_central_image_pop_15px_rad
z_central_image_pop_10px_rad
z_central_image_pop_5px_rad
x_coord_y
y_coord_y
i_petro_rad
i_isophotal_area
i_major_axis
i_minor_axis
i_pos_angle
i_peak_surface_brightness
i_ellipticity
i_half_light_radius
i_sersic_index
i_central_image_pop_15px_rad
i_central_image_pop_10px_rad
i_central_image_pop_5px_rad
x_coord_x
y_coord_x
g_petro_rad
g_isophotal_area
g_major_axis
g_minor_axis
g_pos_angle
g_peak_surface_brightness
g_ellipticity
g_half_light_radius
g_sersic_index
g_central_image_pop_15px_rad
g_central_image_pop_10px_rad
g_central_image_pop_5px_rad
x_coord_y
y_

In [16]:
with h5py.File(file_dir + new_file_name, 'a') as f:
    print(list(f.keys()))

['coord', 'dec', 'g_central_image_pop_10px_rad', 'g_central_image_pop_15px_rad', 'g_central_image_pop_5px_rad', 'g_cmodel_mag', 'g_cmodel_magsigma', 'g_ellipticity', 'g_half_light_radius', 'g_isophotal_area', 'g_major_axis', 'g_minor_axis', 'g_peak_surface_brightness', 'g_petro_rad', 'g_pos_angle', 'g_sersic_index', 'i_central_image_pop_10px_rad', 'i_central_image_pop_15px_rad', 'i_central_image_pop_5px_rad', 'i_cmodel_mag', 'i_cmodel_magsigma', 'i_ellipticity', 'i_half_light_radius', 'i_isophotal_area', 'i_major_axis', 'i_minor_axis', 'i_peak_surface_brightness', 'i_petro_rad', 'i_pos_angle', 'i_sersic_index', 'image', 'object_id', 'r_central_image_pop_10px_rad', 'r_central_image_pop_15px_rad', 'r_central_image_pop_5px_rad', 'r_cmodel_mag', 'r_cmodel_magsigma', 'r_ellipticity', 'r_half_light_radius', 'r_isophotal_area', 'r_major_axis', 'r_minor_axis', 'r_peak_surface_brightness', 'r_petro_rad', 'r_pos_angle', 'r_sersic_index', 'ra', 'skymap_id', 'specz_dec', 'specz_flag_homogeneous', 

In [17]:
# make this for the 5 bands?
# come up with names for the shape parameters if needed
# if this goes in the data, make others have input on this

In [18]:
# ISOAREA_IMAGE -> isophotal_area
# A_IMAGE -> major_axis
# B_IMAGE -> minor_axis
# FLUX_RADIUS -> half_light_rad
# PETRO_RADIUS -> petro_rad
# THETA_WORLD -> pos_angle
# SPHEROID_SERSICN -> sersic_index
# ELLIPTICITY -> ellipticity
# MU_MAX -> peak_surfance_brightness